# Libraries used

Running Kernel3.9.13 base anaconda

pip install squarify
pip install yellowbrick
pip install plotly
pip install seaborn
pip install lazypredict
pip install pandas_profiling

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns  
import matplotlib.pyplot as plt
import plotly.io as pio
import squarify #treemap
import os
import matplotlib
import warnings

#to enable the inline plotting
%matplotlib inline 

import warnings
warnings.filterwarnings('ignore')

sns.set_style("darkgrid")

In [2]:
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import classification_report

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.preprocessing import StandardScaler


from scipy.stats import normaltest

from pandas_profiling import ProfileReport

from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import ClassPredictionError
from yellowbrick.style.palettes import PALETTES, SEQUENCES, color_palette

import lazypredict

warnings.simplefilter(action='ignore', category=FutureWarning)

# Functions


In [3]:
# Function for EDA. Using the display() function to have  well-formatted tables. We are mainly using pandas to explore the datasets

def dataset_description(df_target):

    print('This is the Dataset shape: %s\n' % (df_target.shape, ))
    print('Dataset columns: %s\n' % df_target.columns)

    print('\nColumns description:\n')
    display(df_target.info())
    display(df_target.describe())  # describe the dataset

    print('\nNull values:\n')
    display(df_target.isnull().sum())  # Identify null values

#function performing a quick check on df_inspection with a function to have best of pandas functions separated by a line
def quick_check(dataframe):
    print('First 5 rows %s\n')
    print(dataframe.head(2))
    print("=====================================")
    print('Dataframe shape %s\n')
    print(dataframe.shape)
    print("=====================================")
    print('Dataframe describe categorical %s\n')
    print(dataframe.describe(include=['O']))
    print("=====================================")
    print('Dataframe null values %s\n')
    print(dataframe.isnull().sum())
    print("=====================================")
    print('Dataframe value counts %s\n')
    print(dataframe.value_counts())
    print("=====================================")

def stats(dataframe):
    print('Dataframe correlation %s\n')
    print(dataframe.corr())
    print("=====================================")
    print('Dataframe covariance %s\n')
    print(dataframe.cov())
    print("=====================================")
    print('Dataframe skew %s\n')
    print(dataframe.skew())
    print("=====================================")
    print('Dataframe kurtosis %s\n')
    print(dataframe.kurt())
    print("=====================================")

#create a function to normalize characters from a dataset's column in Spanish
def normalize_characters(df, column):
    df[column] = df[column].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    df[column] = df[column].str.lower()
    df[column] = df[column].str.replace('á', 'a')
    df[column] = df[column].str.replace('é', 'e')
    df[column] = df[column].str.replace('í', 'i')
    df[column] = df[column].str.replace('ó', 'o')
    df[column] = df[column].str.replace('ú', 'u')
    df[column] = df[column].str.replace('ñ', 'n')
    df[column] = df[column].str.replace('ü', 'u')
    df[column] = df[column].str.replace('ç', 'c')
    df[column] = df[column].str.replace('(', '')
    df[column] = df[column].str.replace(')', '')
    df[column] = df[column].str.replace('\'', '')
    df[column] = df[column].str.replace('´', '')
    df[column] = df[column].str.replace('`', '')
    df[column] = df[column].str.replace('’', '')
    return df

In [4]:
#show all print outputs when using a function
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# 2. Data Collection and Understanding 

## Network file

The network.csv file contains a static picture of the gas pipeline network. Every row corresponds to a pipe and has a unique PipeId identifier. The table has 1.446.529 pipes. 

The columns describe relevant features of each pipe. The complete list is: 

PipeId - unique identifier for the pipe 

Province - Spanish province where the pipe is located 

Town - Town or city where the pipe is located 

YearBuilt - Year in which the pipe was built and installed 

Material - Material in which the pipe is built 

GasType - Type of gas that runs through the pipe 

Diameter - diameter of the pipe 

Length - Length of the pipe 

Pressure - Pressure of the gas that runs through the pipe (bar) 

NumConnections - Number of connections (external). 

NumConnectionsUnder - Number of connections (internal and buried) 

BoolBridle  - Whether the pipe is bridled (True) or welded (False) 

## Inspection file

MaintenanceId - unique identifier for the inspection operation 

InspectionYear - year in which the inspection took place 

InspectionDate - date in which the inspection took place 

MonthsLastRev - number of months elapsed since the last previous inspection. 

Severity - Severity of the damage found (1: most severe, 3: least severe) 

Incidence - Boolean whether an incident was found on the revision (1) or not (0). 

# Merging datasets

In [5]:
#point to the folder where the data is stored
os.chdir(r"C:\Users\pedro\datathon")

# Loading inspection data
df_inspection = pd.read_csv('inspections.csv')

# Loading network data
df_network = pd.read_csv('network.csv')

In [6]:
quick_check(df_inspection)


First 5 rows %s

      PipeId MaintenanceId  InspectionYear InspectionDate  MonthsLastRev  \
0  189311802  ZRV-00001972            2010     2010-10-01             24   
1  188889914  ZRV-00001406            2010     2010-10-01             22   

   Severity  Incidence  
0         4          0  
1         4          0  
Dataframe shape %s

(6345344, 7)
Dataframe describe categorical %s

       MaintenanceId InspectionDate
count        6345344        6345344
unique          4179           3565
top     ZRV-00002121     2014-05-05
freq           21773           5832
Dataframe null values %s

PipeId            0
MaintenanceId     0
InspectionYear    0
InspectionDate    0
MonthsLastRev     0
Severity          0
Incidence         0
dtype: int64
Dataframe value counts %s

PipeId     MaintenanceId  InspectionYear  InspectionDate  MonthsLastRev  Severity  Incidence
489616     ZRV-00003385   2013            2013-05-08      16             4         0            1
256852211  ZRV-00002129   2016    

In [7]:
quick_check(df_network)

First 5 rows %s

   PipeId     Province                   Town  YearBuilt Material  \
0  489616  Ciudad Real  Torralba de Calatrava       2012       PE   
1  489645  Ciudad Real  Torralba de Calatrava       2012       PE   

       GasType  Diameter  Length  Pressure  NumConnections  \
0  Gas natural      63.0   7.792       0.4               0   
1  Gas natural      90.0   2.080       0.4               0   

   NumConnectionsUnder  BoolBridle  
0                    0       False  
1                    0       False  
Dataframe shape %s

(1446539, 12)
Dataframe describe categorical %s

         Province     Town Material      GasType
count     1446539  1446539  1446539      1446539
unique         38     1972       11            2
top     Barcelona   Madrid       PE  Gas natural
freq       389307    76666  1242845      1388397
Dataframe null values %s

PipeId                 0
Province               0
Town                   0
YearBuilt              0
Material               0
GasType     

In [8]:
stats(df_inspection)

Dataframe correlation %s

                  PipeId  InspectionYear  MonthsLastRev  Severity  Incidence
PipeId          1.000000       -0.123868       0.048799  0.017614  -0.021449
InspectionYear -0.123868        1.000000       0.117585  0.001188  -0.000359
MonthsLastRev   0.048799        0.117585       1.000000  0.011268  -0.013749
Severity        0.017614        0.001188       0.011268  1.000000  -0.904174
Incidence      -0.021449       -0.000359      -0.013749 -0.904174   1.000000
Dataframe covariance %s

                      PipeId  InspectionYear  MonthsLastRev       Severity  \
PipeId          1.294358e+16   -4.099991e+07   2.754554e+07  129283.543032   
InspectionYear -4.099991e+07    8.464272e+00   1.697315e+00       0.000223   
MonthsLastRev   2.754554e+07    1.697315e+00   2.461659e+01       0.003607   
Severity        1.292835e+05    2.230601e-04   3.606757e-03       0.004162   
Incidence      -1.074542e+05   -4.603098e-05  -3.003968e-03      -0.002569   

                  

In [9]:
stats(df_network)

Dataframe correlation %s

                       PipeId  YearBuilt  Diameter    Length  Pressure  \
PipeId               1.000000  -0.052882  0.133925 -0.013087  0.044928   
YearBuilt           -0.052882   1.000000 -0.155437  0.028727  0.047748   
Diameter             0.133925  -0.155437  1.000000  0.073956  0.194073   
Length              -0.013087   0.028727  0.073956  1.000000  0.105697   
Pressure             0.044928   0.047748  0.194073  0.105697  1.000000   
NumConnections      -0.045010  -0.028777 -0.112723  0.307837 -0.102855   
NumConnectionsUnder -0.010956   0.011157 -0.007065  0.017895 -0.002610   
BoolBridle          -0.091219  -0.176955  0.089100 -0.022166 -0.065808   

                     NumConnections  NumConnectionsUnder  BoolBridle  
PipeId                    -0.045010            -0.010956   -0.091219  
YearBuilt                 -0.028777             0.011157   -0.176955  
Diameter                  -0.112723            -0.007065    0.089100  
Length                 

In [10]:
#merge both dataframes
df_combined = pd.merge(df_inspection, df_network, on="PipeId")

In [11]:
df_combined.head(1)

,PipeId,MaintenanceId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle
0,189311802,ZRV-00001972,2010,2010-10-01,24,4,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.0,0,0,False


In [12]:
#creating a subset for altering the dataset after initial EDA
df_combined_mod = df_combined.copy()

In [13]:
df_combined_mod.head(1)


,PipeId,MaintenanceId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle
0,189311802,ZRV-00001972,2010,2010-10-01,24,4,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.0,0,0,False


In [14]:
df_combined_mod.columns

Index(['PipeId', 'MaintenanceId', 'InspectionYear', 'InspectionDate',
       'MonthsLastRev', 'Severity', 'Incidence', 'Province', 'Town',
       'YearBuilt', 'Material', 'GasType', 'Diameter', 'Length', 'Pressure',
       'NumConnections', 'NumConnectionsUnder', 'BoolBridle'],
      dtype='object')

# Data Wrangling on combined dataset

As a result of merging both datasets we now have pipeline duplicates per each maintenace_id operation. Before getrtng rid of the duplicates, we want to engineer some metrics interesting to the model such as number of operations, number of incidents and average risk based on severity*incidence

In [15]:
#show all columns in pandas
pd.set_option('display.max_columns', None)

In [16]:
#create a new column counting the number of inspections (MaintenanceId) per PipeId
df_combined_mod['Inspections'] = df_combined_mod.groupby('PipeId')['MaintenanceId'].transform('count')
#aggregate the number of Incidents per pipe in a new column and place it in the fourth position
df_combined_mod['No_Incidents'] = df_combined_mod.groupby('PipeId')['Incidence'].transform('sum')
#place the new columns in the third position
cols = list(df_combined_mod.columns.values)
cols.pop(cols.index('No_Incidents'))
df_combined_mod = df_combined_mod[['PipeId', 'MaintenanceId', 'Inspections', 'No_Incidents', 'InspectionYear', 'InspectionDate',
       'MonthsLastRev', 'Severity', 'Incidence', 'Province', 'Town',
       'YearBuilt', 'Material', 'GasType', 'Diameter', 'Length', 'Pressure',
       'NumConnections', 'NumConnectionsUnder', 'BoolBridle']]
#show head of rows only where No_Incidents is greater than 0
df_combined_mod[df_combined_mod['No_Incidents'] > 2].head(10)


'No_Incidents'

,PipeId,MaintenanceId,Inspections,No_Incidents,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle
140753,191522404,ZRV-00003686,5,3,2010,2010-10-19,19,4,0,Barcelona,Igualada,1988,AO,Gas natural,304.8,220.382828,0.025,0,0,True
140754,191522404,ZRV-00003686,5,3,2012,2012-11-02,25,1,1,Barcelona,Igualada,1988,AO,Gas natural,304.8,220.382828,0.025,0,0,True
140755,191522404,ZRV-00003686,5,3,2014,2014-10-01,23,4,0,Barcelona,Igualada,1988,AO,Gas natural,304.8,220.382828,0.025,0,0,True
140756,191522404,ZRV-00003686,5,3,2016,2016-11-11,25,2,1,Barcelona,Igualada,1988,AO,Gas natural,304.8,220.382828,0.025,0,0,True
140757,191522404,ZRV-00003686,5,3,2018,2018-11-05,24,2,1,Barcelona,Igualada,1988,AO,Gas natural,304.8,220.382828,0.025,0,0,True
272048,287737413,ZRV-00003836,6,3,2010,2010-11-05,18,4,0,Madrid,Madrid,1988,FD,Gas natural,200.0,107.866000,0.025,8,0,True
272049,287737413,ZRV-00003836,6,3,2012,2012-09-17,22,3,1,Madrid,Madrid,1988,FD,Gas natural,200.0,107.866000,0.025,8,0,True
272050,287737413,ZRV-00003836,6,3,2014,2014-06-10,21,4,0,Madrid,Madrid,1988,FD,Gas natural,200.0,107.866000,0.025,8,0,True
272051,287737413,ZRV-00003836,6,3,2016,2016-03-29,21,4,0,Madrid,Madrid,1988,FD,Gas natural,200.0,107.866000,0.025,8,0,True
272052,287737413,ZRV-00003836,6,3,2018,2018-02-02,23,2,1,Madrid,Madrid,1988,FD,Gas natural,200.0,107.866000,0.025,8,0,True


In [17]:
df_combined_mod.columns

Index(['PipeId', 'MaintenanceId', 'Inspections', 'No_Incidents',
       'InspectionYear', 'InspectionDate', 'MonthsLastRev', 'Severity',
       'Incidence', 'Province', 'Town', 'YearBuilt', 'Material', 'GasType',
       'Diameter', 'Length', 'Pressure', 'NumConnections',
       'NumConnectionsUnder', 'BoolBridle'],
      dtype='object')

In [18]:
#create a column named average_severity that calculates the average severity per pipe when Incidence is 1
df_combined_mod['average_severity'] = df_combined_mod.groupby('PipeId')['Severity'].transform('mean')
#show head of rows only where mean has a decimal value
df_combined_mod[df_combined_mod['average_severity'] % 1 != 0].head(10)
#place the new column in 7th position
cols = list(df_combined_mod.columns.values)
cols.pop(cols.index('average_severity'))
df_combined_mod = df_combined_mod[['PipeId', 'MaintenanceId', 'Inspections', 'No_Incidents', 
    'InspectionYear', 'InspectionDate', 'average_severity', 'MonthsLastRev', 'Severity', 'Incidence', 'Province',
    'Town','YearBuilt', 'Material', 'GasType', 'Diameter', 'Length', 'Pressure', 'NumConnections',
    'NumConnectionsUnder', 'BoolBridle']]


,PipeId,MaintenanceId,Inspections,No_Incidents,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,average_severity
108,188889938,ZRV-00001406,6,1,2010,2010-10-01,22,4,0,Barcelona,Barcelona,1999,PE,Gas natural,160.0,77.024,0.025,5,0,False,3.833333
109,188889938,ZRV-00001406,6,1,2011,2011-11-17,13,4,0,Barcelona,Barcelona,1999,PE,Gas natural,160.0,77.024,0.025,5,0,False,3.833333
110,188889938,ZRV-00001406,6,1,2013,2013-11-12,24,4,0,Barcelona,Barcelona,1999,PE,Gas natural,160.0,77.024,0.025,5,0,False,3.833333
111,188889938,ZRV-00001406,6,1,2015,2015-11-06,24,3,1,Barcelona,Barcelona,1999,PE,Gas natural,160.0,77.024,0.025,5,0,False,3.833333
112,188889938,ZRV-00001406,6,1,2017,2017-11-23,24,4,0,Barcelona,Barcelona,1999,PE,Gas natural,160.0,77.024,0.025,5,0,False,3.833333
113,188889938,ZRV-00001406,6,1,2019,2019-11-21,24,4,0,Barcelona,Barcelona,1999,PE,Gas natural,160.0,77.024,0.025,5,0,False,3.833333
186,188909811,ZRV-00001406,6,1,2010,2010-10-01,22,4,0,Barcelona,Barcelona,1999,PE,Gas natural,200.0,34.436,0.025,4,0,False,3.500000
187,188909811,ZRV-00001406,6,1,2011,2011-11-17,13,4,0,Barcelona,Barcelona,1999,PE,Gas natural,200.0,34.436,0.025,4,0,False,3.500000
188,188909811,ZRV-00001406,6,1,2013,2013-11-11,24,4,0,Barcelona,Barcelona,1999,PE,Gas natural,200.0,34.436,0.025,4,0,False,3.500000
189,188909811,ZRV-00001406,6,1,2015,2015-11-03,24,4,0,Barcelona,Barcelona,1999,PE,Gas natural,200.0,34.436,0.025,4,0,False,3.500000


'average_severity'

In [19]:
#create a column taking average severity and incidence multiplying them and naming it as risk_(s*i)
df_combined_mod['Risk_S*I'] = df_combined_mod['average_severity'] * df_combined_mod['No_Incidents'] 
#position the new column in the 7th position
cols = list(df_combined_mod.columns.values)
cols.pop(cols.index('Risk_S*I'))
df_combined_mod = df_combined_mod[['PipeId', 'MaintenanceId', 'Inspections', 'No_Incidents', 'Risk_S*I',
    'InspectionYear', 'InspectionDate', 'average_severity', 'MonthsLastRev', 'Severity', 'Incidence', 'Province',
    'Town','YearBuilt', 'Material', 'GasType', 'Diameter', 'Length', 'Pressure', 'NumConnections',
    'NumConnectionsUnder', 'BoolBridle']]

'Risk_S*I'

In [20]:
#show head of rows only where Risk_S*I is greater than 0
df_combined_mod[df_combined_mod['Risk_S*I'] > 0].head(10)

,PipeId,MaintenanceId,Inspections,No_Incidents,Risk_S*I,InspectionYear,InspectionDate,average_severity,MonthsLastRev,Severity,Incidence,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle
108,188889938,ZRV-00001406,6,1,3.833333,2010,2010-10-01,3.833333,22,4,0,Barcelona,Barcelona,1999,PE,Gas natural,160.0,77.024,0.025,5,0,False
109,188889938,ZRV-00001406,6,1,3.833333,2011,2011-11-17,3.833333,13,4,0,Barcelona,Barcelona,1999,PE,Gas natural,160.0,77.024,0.025,5,0,False
110,188889938,ZRV-00001406,6,1,3.833333,2013,2013-11-12,3.833333,24,4,0,Barcelona,Barcelona,1999,PE,Gas natural,160.0,77.024,0.025,5,0,False
111,188889938,ZRV-00001406,6,1,3.833333,2015,2015-11-06,3.833333,24,3,1,Barcelona,Barcelona,1999,PE,Gas natural,160.0,77.024,0.025,5,0,False
112,188889938,ZRV-00001406,6,1,3.833333,2017,2017-11-23,3.833333,24,4,0,Barcelona,Barcelona,1999,PE,Gas natural,160.0,77.024,0.025,5,0,False
113,188889938,ZRV-00001406,6,1,3.833333,2019,2019-11-21,3.833333,24,4,0,Barcelona,Barcelona,1999,PE,Gas natural,160.0,77.024,0.025,5,0,False
186,188909811,ZRV-00001406,6,1,3.500000,2010,2010-10-01,3.500000,22,4,0,Barcelona,Barcelona,1999,PE,Gas natural,200.0,34.436,0.025,4,0,False
187,188909811,ZRV-00001406,6,1,3.500000,2011,2011-11-17,3.500000,13,4,0,Barcelona,Barcelona,1999,PE,Gas natural,200.0,34.436,0.025,4,0,False
188,188909811,ZRV-00001406,6,1,3.500000,2013,2013-11-11,3.500000,24,4,0,Barcelona,Barcelona,1999,PE,Gas natural,200.0,34.436,0.025,4,0,False
189,188909811,ZRV-00001406,6,1,3.500000,2015,2015-11-03,3.500000,24,4,0,Barcelona,Barcelona,1999,PE,Gas natural,200.0,34.436,0.025,4,0,False


In [21]:
#create a new column taking Risk_S*I and dividing it by No_Inspections naming it as Risk_S*I/Inspections and placing it in 6th position
df_combined_mod['Risk_S*I/Inspections'] = df_combined_mod['Risk_S*I'] / df_combined_mod['Inspections']
cols = list(df_combined_mod.columns.values)
cols.pop(cols.index('Risk_S*I/Inspections'))
df_combined_mod = df_combined_mod[['PipeId', 'MaintenanceId', 'Inspections', 'Risk_S*I/Inspections', 'No_Incidents', 
    'Risk_S*I', 'average_severity', 'MonthsLastRev', 'Severity', 'Incidence', 'Province',
    'Town','YearBuilt', 'InspectionYear', 'InspectionDate','Material', 'GasType', 'Diameter', 'Length', 'Pressure', 'NumConnections',
    'NumConnectionsUnder', 'BoolBridle']]

#show head of rows only where Risk_S*I/Inspections is greater than 0
df_combined_mod[df_combined_mod['Risk_S*I/Inspections'] > 0].head(5)

'Risk_S*I/Inspections'

,PipeId,MaintenanceId,Inspections,Risk_S*I/Inspections,No_Incidents,Risk_S*I,average_severity,MonthsLastRev,Severity,Incidence,Province,Town,YearBuilt,InspectionYear,InspectionDate,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle
108,188889938,ZRV-00001406,6,0.638889,1,3.833333,3.833333,22,4,0,Barcelona,Barcelona,1999,2010,2010-10-01,PE,Gas natural,160.0,77.024,0.025,5,0,False
109,188889938,ZRV-00001406,6,0.638889,1,3.833333,3.833333,13,4,0,Barcelona,Barcelona,1999,2011,2011-11-17,PE,Gas natural,160.0,77.024,0.025,5,0,False
110,188889938,ZRV-00001406,6,0.638889,1,3.833333,3.833333,24,4,0,Barcelona,Barcelona,1999,2013,2013-11-12,PE,Gas natural,160.0,77.024,0.025,5,0,False
111,188889938,ZRV-00001406,6,0.638889,1,3.833333,3.833333,24,3,1,Barcelona,Barcelona,1999,2015,2015-11-06,PE,Gas natural,160.0,77.024,0.025,5,0,False
112,188889938,ZRV-00001406,6,0.638889,1,3.833333,3.833333,24,4,0,Barcelona,Barcelona,1999,2017,2017-11-23,PE,Gas natural,160.0,77.024,0.025,5,0,False


In [22]:
#transform Inspection Date to datetime format
df_combined_mod['InspectionDate'] = pd.to_datetime(df_combined_mod['InspectionDate'])


In [23]:
#creating a new column named probability with No_Incidents divided by Inspections
df_combined_mod['Probability_incidence'] = df_combined_mod['No_Incidents'] / df_combined_mod['Inspections']
#place column in 3rd position
cols = list(df_combined_mod.columns.values)
cols.pop(cols.index('Probability_incidence'))
df_combined_mod = df_combined_mod[['PipeId', 'MaintenanceId', 'Inspections', 'Probability_incidence', 'Risk_S*I/Inspections', 
    'No_Incidents', 'Risk_S*I', 'average_severity', 'MonthsLastRev', 'Severity', 'Incidence', 'Province',
    'Town','YearBuilt', 'InspectionYear', 'InspectionDate','Material', 'GasType', 'Diameter', 'Length', 'Pressure',
    'NumConnections', 'NumConnectionsUnder', 'BoolBridle']]
#show head of rows only where Probability is greater than 0
df_combined_mod[df_combined_mod['Probability_incidence'] > 0].head(5)

'Probability_incidence'

,PipeId,MaintenanceId,Inspections,Probability_incidence,Risk_S*I/Inspections,No_Incidents,Risk_S*I,average_severity,MonthsLastRev,Severity,Incidence,Province,Town,YearBuilt,InspectionYear,InspectionDate,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle
108,188889938,ZRV-00001406,6,0.166667,0.638889,1,3.833333,3.833333,22,4,0,Barcelona,Barcelona,1999,2010,2010-10-01,PE,Gas natural,160.0,77.024,0.025,5,0,False
109,188889938,ZRV-00001406,6,0.166667,0.638889,1,3.833333,3.833333,13,4,0,Barcelona,Barcelona,1999,2011,2011-11-17,PE,Gas natural,160.0,77.024,0.025,5,0,False
110,188889938,ZRV-00001406,6,0.166667,0.638889,1,3.833333,3.833333,24,4,0,Barcelona,Barcelona,1999,2013,2013-11-12,PE,Gas natural,160.0,77.024,0.025,5,0,False
111,188889938,ZRV-00001406,6,0.166667,0.638889,1,3.833333,3.833333,24,3,1,Barcelona,Barcelona,1999,2015,2015-11-06,PE,Gas natural,160.0,77.024,0.025,5,0,False
112,188889938,ZRV-00001406,6,0.166667,0.638889,1,3.833333,3.833333,24,4,0,Barcelona,Barcelona,1999,2017,2017-11-23,PE,Gas natural,160.0,77.024,0.025,5,0,False


# Calculating aprox gas leakage for case study
## Revisit whenever team has come down with a factor!

### JUAN: Risk Matrix
    Alto riesgo alta probabilidad Bajo riesgo baja probabilidad

Before narrowing down the dataset, we want to take an ssumption in the case there has been an incidence, to take half of the months since last inspection and multiply by a factor to estimate losses and hence potential savings

In [24]:
#create a new column named aprox_gas leakage taking PipeId with Incidence = 1 and multyplyig a factor of to MonthsLastRev/2
df_combined_mod['leakage_estimate_factor'] = df_combined_mod['Risk_S*I/Inspections'] * (df_combined_mod['MonthsLastRev']/2)
#position the new column in the 6th position
cols = list(df_combined_mod.columns.values)
cols.pop(cols.index('leakage_estimate_factor'))
df_combined_mod = df_combined_mod[['PipeId', 'MaintenanceId', 'Inspections', 'No_Incidents', 'Risk_S*I/Inspections','leakage_estimate_factor','InspectionYear', 'InspectionDate', 'MonthsLastRev', 'Risk_S*I',
    'Severity','Incidence', 'Province', 'Town', 'YearBuilt', 'Material', 'GasType','Diameter', 'Length', 'Pressure', 'NumConnections', 'NumConnectionsUnder', 'BoolBridle']]
df_combined_mod.head()

'leakage_estimate_factor'

,PipeId,MaintenanceId,Inspections,No_Incidents,Risk_S*I/Inspections,leakage_estimate_factor,InspectionYear,InspectionDate,MonthsLastRev,Risk_S*I,Severity,Incidence,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle
0,189311802,ZRV-00001972,6,0,0.0,0.0,2010,2010-10-01,24,0.0,4,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.0,0,0,False
1,189311802,ZRV-00001972,6,0,0.0,0.0,2012,2012-10-01,24,0.0,4,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.0,0,0,False
2,189311802,ZRV-00001972,6,0,0.0,0.0,2014,2014-10-08,24,0.0,4,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.0,0,0,False
3,189311802,ZRV-00001972,6,0,0.0,0.0,2016,2016-10-14,24,0.0,4,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.0,0,0,False
4,189311802,ZRV-00001972,6,0,0.0,0.0,2018,2018-10-09,24,0.0,4,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.0,0,0,False


# Narrowing combined dataset

Now that we have extracted the interesting information from combining the dataset, we will further transform it to only include 1 pipeID and proceed with further data wrangling. We eliminate the maintenaceId for now

In [25]:
#keep only one PipeId per row
df_combined_mod = df_combined_mod.drop_duplicates(subset=['PipeId'], keep='first')
df_combined_mod.head()

,PipeId,MaintenanceId,Inspections,No_Incidents,Risk_S*I/Inspections,leakage_estimate_factor,InspectionYear,InspectionDate,MonthsLastRev,Risk_S*I,Severity,Incidence,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle
0,189311802,ZRV-00001972,6,0,0.0,0.0,2010,2010-10-01,24,0.0,4,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False
6,188889914,ZRV-00001406,6,0,0.0,0.0,2010,2010-10-01,22,0.0,4,0,Barcelona,Barcelona,1999,PE,Gas natural,160.0,1.292,0.025,0,0,False
12,188889907,ZRV-00001406,6,0,0.0,0.0,2010,2010-10-01,22,0.0,4,0,Barcelona,Barcelona,1992,PE,Gas natural,200.0,28.193,0.025,1,0,False
18,188673355,ZRV-00001406,6,0,0.0,0.0,2010,2010-10-01,22,0.0,4,0,Barcelona,Barcelona,1998,PE,Gas natural,200.0,66.259,0.025,5,0,False
24,188558922,ZRV-00001406,6,0,0.0,0.0,2010,2010-10-01,22,0.0,4,0,Barcelona,Barcelona,1999,PE,Gas natural,160.0,0.810,0.025,0,0,False


In [26]:
#removing MaintenanceId column
df_combined_mod = df_combined_mod.drop(['MaintenanceId'], axis=1)

# Data Wrangling
Start of cleaning and organizing a dataset

In [27]:
df_combined['BoolBridle'].describe()

count     6345344
unique          2
top         False
freq      5967747
Name: BoolBridle, dtype: object

In [28]:
# Converting Boolbride into  boolean variable
def boolbridle(x):
    return 1 if x == 'True' else 0

In [29]:
# Apply function on dataset
df_combined['BoolBridle'] = df_combined['BoolBridle'].apply(lambda x: boolbridle(x))

In [30]:
#using custom made function to describe the dataset
dataset_description(df_combined_mod)

This is the Dataset shape: (1427854, 22)

Dataset columns: Index(['PipeId', 'Inspections', 'No_Incidents', 'Risk_S*I/Inspections',
       'leakage_estimate_factor', 'InspectionYear', 'InspectionDate',
       'MonthsLastRev', 'Risk_S*I', 'Severity', 'Incidence', 'Province',
       'Town', 'YearBuilt', 'Material', 'GasType', 'Diameter', 'Length',
       'Pressure', 'NumConnections', 'NumConnectionsUnder', 'BoolBridle'],
      dtype='object')


Columns description:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1427854 entries, 0 to 6345343
Data columns (total 22 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   PipeId                   1427854 non-null  int64         
 1   Inspections              1427854 non-null  int64         
 2   No_Incidents             1427854 non-null  int64         
 3   Risk_S*I/Inspections     1427854 non-null  float64       
 4   leakage_estimate_factor  1427854 non

None

,PipeId,Inspections,No_Incidents,Risk_S*I/Inspections,leakage_estimate_factor,InspectionYear,MonthsLastRev,Risk_S*I,Severity,Incidence,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder
count,1.427854e+06,1.427854e+06,1.427854e+06,1.427854e+06,1.427854e+06,1.427854e+06,1.427854e+06,1.427854e+06,1.427854e+06,1.427854e+06,1.427854e+06,1.427854e+06,1.427854e+06,1.427854e+06,1.427854e+06,1.427854e+06
mean,1.872945e+08,4.443973e+00,8.633936e-03,8.957451e-03,9.442791e-02,2.012496e+03,2.294084e+01,3.070590e-02,3.995605e+00,3.420518e-03,2.002395e+03,1.166339e+02,3.679221e+01,3.066029e+00,9.651148e-01,3.431723e-04
std,1.208057e+08,1.378694e+00,9.931492e-02,1.152558e-01,1.394785e+00,2.311752e+00,6.303686e+00,3.526224e-01,8.308406e-02,5.838510e-02,1.206357e+01,5.788525e+01,7.815416e+01,6.911702e+00,2.158412e+00,2.018631e-02
min,4.896160e+05,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.010000e+03,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.901000e+03,1.000000e+01,0.000000e+00,2.500000e-02,0.000000e+00,0.000000e+00
25%,5.265259e+07,4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.011000e+03,2.200000e+01,0.000000e+00,4.000000e+00,0.000000e+00,1.998000e+03,7.620000e+01,3.500000e+00,1.000000e-01,0.000000e+00,0.000000e+00
50%,1.897264e+08,5.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.012000e+03,2.400000e+01,0.000000e+00,4.000000e+00,0.000000e+00,2.004000e+03,1.100000e+02,1.359800e+01,1.500000e-01,0.000000e+00,0.000000e+00
75%,2.908250e+08,5.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.013000e+03,2.400000e+01,0.000000e+00,4.000000e+00,0.000000e+00,2.009000e+03,1.600000e+02,4.437775e+01,4.000000e+00,1.000000e+00,0.000000e+00
max,4.519953e+08,1.100000e+01,5.000000e+00,3.000000e+00,1.980000e+02,2.021000e+03,1.320000e+02,1.500000e+01,4.000000e+00,1.000000e+00,2.050000e+03,6.096000e+02,2.610094e+04,8.000000e+01,8.800000e+01,4.000000e+00



Null values:



PipeId                     0
Inspections                0
No_Incidents               0
Risk_S*I/Inspections       0
leakage_estimate_factor    0
InspectionYear             0
InspectionDate             0
MonthsLastRev              0
Risk_S*I                   0
Severity                   0
Incidence                  0
Province                   0
Town                       0
YearBuilt                  0
Material                   0
GasType                    0
Diameter                   0
Length                     0
Pressure                   0
NumConnections             0
NumConnectionsUnder        0
BoolBridle                 0
dtype: int64

In [31]:
#showing head
df_combined_mod.head(1)

,PipeId,Inspections,No_Incidents,Risk_S*I/Inspections,leakage_estimate_factor,InspectionYear,InspectionDate,MonthsLastRev,Risk_S*I,Severity,Incidence,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle
0,189311802,6,0,0.0,0.0,2010,2010-10-01,24,0.0,4,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.0,0,0,False


In [32]:
#Extract the day of the week from InspectionDate and place it in 5th position
df_combined_mod['InspectionDay'] = df_combined_mod['InspectionDate'].dt.day_name()
cols = list(df_combined_mod.columns.values)
cols.pop(cols.index('InspectionDay'))
df_combined_mod = df_combined_mod[['PipeId', 'Inspections', 'No_Incidents', 'Risk_S*I/Inspections','leakage_estimate_factor','InspectionDay',
    'InspectionYear', 'InspectionDate', 'MonthsLastRev', 'Risk_S*I','Severity','Incidence', 'Province', 'Town', 'YearBuilt', 'Material', 'GasType',
    'Diameter', 'Length', 'Pressure', 'NumConnections', 'NumConnectionsUnder', 'BoolBridle']]
df_combined_mod.head(1)


'InspectionDay'

,PipeId,Inspections,No_Incidents,Risk_S*I/Inspections,leakage_estimate_factor,InspectionDay,InspectionYear,InspectionDate,MonthsLastRev,Risk_S*I,Severity,Incidence,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle
0,189311802,6,0,0.0,0.0,Friday,2010,2010-10-01,24,0.0,4,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.0,0,0,False


In [33]:
#display possible values for week_day column and how many times they appear
df_combined_mod['InspectionDay'].value_counts()

Tuesday      298009
Wednesday    294942
Monday       288238
Thursday     283746
Friday       226550
Saturday      20088
Sunday        16281
Name: InspectionDay, dtype: int64

In [34]:
#hot encode GasType column in the df_combined_mod dataframe subset
df_combined_mod = pd.get_dummies(df_combined_mod, columns=['GasType'], prefix = ['GasType'])

In [35]:
#delete GasType_Gas propano column
df_combined_mod = df_combined_mod.drop(['GasType_Gas propano'], axis=1)

In [36]:
#change name of GasType_Gas natural column to gas_natural
df_combined_mod = df_combined_mod.rename(columns={'GasType_Gas natural': 'gas_natural'})
df_combined_mod.head(1)

,PipeId,Inspections,No_Incidents,Risk_S*I/Inspections,leakage_estimate_factor,InspectionDay,InspectionYear,InspectionDate,MonthsLastRev,Risk_S*I,Severity,Incidence,Province,Town,YearBuilt,Material,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,gas_natural
0,189311802,6,0,0.0,0.0,Friday,2010,2010-10-01,24,0.0,4,0,Barcelona,Sentmenat,2001,PE,160.0,117.831,4.0,0,0,False,1


# Adding a new dataset
We want to extract value of two categorical variables, Town and Province but the way we have them now they are useless.

We will add a new dataset to join and extract the surface of each town as well as the comunidad autonoma to group by accordingly in another column the number of towns and afterwards hot encode


In [37]:
#count towns populating Town column
df_combined_mod['Town'].value_counts()
df_combined_mod['Town'].unique()

Madrid                  75616
Barcelona               58408
Valencia                25405
Sevilla                 22491
Terrassa                16573
                        ...  
Duruelo de la Sierra        1
Rodeiro                     1
Santurdejo                  1
Bañares                     1
Vecinos                     1
Name: Town, Length: 1960, dtype: int64

array(['Sentmenat', 'Barcelona', 'Montcada I Reixach', ..., 'Albillos',
       'El Puente del Arzobispo', 'Navahermosa'], dtype=object)

In [38]:
#point to the folder where the data is stored
os.chdir(r"C:\Users\pedro\datathon\base\complementary_datasets")

# Loading combined_mod dataset
df_mun = pd.read_excel('list-mun-2012.xls' , sheet_name='list-mun')

df_population= pd.read_excel('pobmun20.xls')


In [39]:
df_mun.head(1)

,codine,Municipio,Superficie,Capitalidad,Año,CA,Autonomía,CP,Provincia,CPJ,Partido_Judicial
0,1001,Alegría-Dulantzi,19.95,Alegría-Dulantzi,NaN,16,País Vasco,1,Araba/Álava,102,Vitoria-Gasteiz


In [40]:
df_population.head(1)

,PROVINCIA,NOMBRE,POB20,HOMBRES,MUJERES
0,Albacete,Abengibre,761,365,396


In [41]:
normalize_characters(df_population, 'PROVINCIA')

,PROVINCIA,NOMBRE,POB20,HOMBRES,MUJERES
0,albacete,Abengibre,761,365,396
1,albacete,Alatoz,506,283,223
2,albacete,Albacete,174336,85214,89122
3,albacete,Albatana,679,351,328
4,albacete,Alborea,664,346,318
...,...,...,...,...,...
8126,zaragoza,"Vilueña, La",62,31,31
8127,zaragoza,Vistabella,48,27,21
8128,zaragoza,"Zaida, La",447,229,218
8129,zaragoza,Zaragoza,681877,326103,355774


In [42]:
normalize_characters(df_mun, 'Municipio')

,codine,Municipio,Superficie,Capitalidad,Año,CA,Autonomía,CP,Provincia,CPJ,Partido_Judicial
0,1001,alegria-dulantzi,19.95,Alegría-Dulantzi,NaN,16,País Vasco,1,Araba/Álava,102,Vitoria-Gasteiz
1,1002,amurrio,96.36,Amurrio,NaN,16,País Vasco,1,Araba/Álava,101,Amurrio
2,1003,aramaio,73.27,Ibarra,NaN,16,País Vasco,1,Araba/Álava,102,Vitoria-Gasteiz
3,1004,artziniega,27.45,Artziniega,NaN,16,País Vasco,1,Araba/Álava,101,Amurrio
4,1006,arminon,12.94,Armiñón,NaN,16,País Vasco,1,Araba/Álava,102,Vitoria-Gasteiz
...,...,...,...,...,...,...,...,...,...,...,...
8111,50901,biel,130.73,Biel,NaN,2,Aragón,50,Zaragoza,5005,Ejea de los Caballeros
8112,50902,marracos,16.92,Marracos,1998.0,2,Aragón,50,Zaragoza,5005,Ejea de los Caballeros
8113,50903,villamayor de gallego,89.36,Villamayor ...,2006.0,2,Aragón,50,Zaragoza,5003,Zaragoza
8114,51001,ceuta,19.48,Ceuta,NaN,18,Ceuta (Ciudad de),51,Ceuta,5112,Ceuta


In [43]:
normalize_characters(df_combined_mod, 'Town')

,PipeId,Inspections,No_Incidents,Risk_S*I/Inspections,leakage_estimate_factor,InspectionDay,InspectionYear,InspectionDate,MonthsLastRev,Risk_S*I,Severity,Incidence,Province,Town,YearBuilt,Material,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,gas_natural
0,189311802,6,0,0.0,0.0,Friday,2010,2010-10-01,24,0.0,4,0,Barcelona,sentmenat,2001,PE,160.0,117.831,4.000,0,0,False,1
6,188889914,6,0,0.0,0.0,Friday,2010,2010-10-01,22,0.0,4,0,Barcelona,barcelona,1999,PE,160.0,1.292,0.025,0,0,False,1
12,188889907,6,0,0.0,0.0,Friday,2010,2010-10-01,22,0.0,4,0,Barcelona,barcelona,1992,PE,200.0,28.193,0.025,1,0,False,1
18,188673355,6,0,0.0,0.0,Friday,2010,2010-10-01,22,0.0,4,0,Barcelona,barcelona,1998,PE,200.0,66.259,0.025,5,0,False,1
24,188558922,6,0,0.0,0.0,Friday,2010,2010-10-01,22,0.0,4,0,Barcelona,barcelona,1999,PE,160.0,0.810,0.025,0,0,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6345339,56922448,1,0,0.0,0.0,Thursday,2020,2020-12-31,24,0.0,4,0,Valencia,betera,1993,PE,63.0,1.237,4.000,0,0,False,1
6345340,56907425,1,0,0.0,0.0,Thursday,2020,2020-12-31,24,0.0,4,0,Valencia,betera,2000,PE,160.0,169.732,4.000,1,0,False,1
6345341,7843615,1,0,0.0,0.0,Thursday,2021,2020-12-31,24,0.0,4,0,Barcelona,sabadell,1995,FD,150.0,0.361,0.025,0,0,True,1
6345342,56922456,1,0,0.0,0.0,Thursday,2020,2020-12-31,24,0.0,4,0,Valencia,betera,1993,PE,63.0,23.306,4.000,0,0,False,1


In [44]:
normalize_characters(df_combined_mod, 'Province')


,PipeId,Inspections,No_Incidents,Risk_S*I/Inspections,leakage_estimate_factor,InspectionDay,InspectionYear,InspectionDate,MonthsLastRev,Risk_S*I,Severity,Incidence,Province,Town,YearBuilt,Material,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,gas_natural
0,189311802,6,0,0.0,0.0,Friday,2010,2010-10-01,24,0.0,4,0,barcelona,sentmenat,2001,PE,160.0,117.831,4.000,0,0,False,1
6,188889914,6,0,0.0,0.0,Friday,2010,2010-10-01,22,0.0,4,0,barcelona,barcelona,1999,PE,160.0,1.292,0.025,0,0,False,1
12,188889907,6,0,0.0,0.0,Friday,2010,2010-10-01,22,0.0,4,0,barcelona,barcelona,1992,PE,200.0,28.193,0.025,1,0,False,1
18,188673355,6,0,0.0,0.0,Friday,2010,2010-10-01,22,0.0,4,0,barcelona,barcelona,1998,PE,200.0,66.259,0.025,5,0,False,1
24,188558922,6,0,0.0,0.0,Friday,2010,2010-10-01,22,0.0,4,0,barcelona,barcelona,1999,PE,160.0,0.810,0.025,0,0,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6345339,56922448,1,0,0.0,0.0,Thursday,2020,2020-12-31,24,0.0,4,0,valencia,betera,1993,PE,63.0,1.237,4.000,0,0,False,1
6345340,56907425,1,0,0.0,0.0,Thursday,2020,2020-12-31,24,0.0,4,0,valencia,betera,2000,PE,160.0,169.732,4.000,1,0,False,1
6345341,7843615,1,0,0.0,0.0,Thursday,2021,2020-12-31,24,0.0,4,0,barcelona,sabadell,1995,FD,150.0,0.361,0.025,0,0,True,1
6345342,56922456,1,0,0.0,0.0,Thursday,2020,2020-12-31,24,0.0,4,0,valencia,betera,1993,PE,63.0,23.306,4.000,0,0,False,1


In [45]:
#join df_combined_mod and df_mun on Town column and Municipio column
df_combined_mod = pd.merge(df_combined_mod, df_mun, how='left', left_on='Town', right_on='Municipio')
df_combined_mod.head(2)

,PipeId,Inspections,No_Incidents,Risk_S*I/Inspections,leakage_estimate_factor,InspectionDay,InspectionYear,InspectionDate,MonthsLastRev,Risk_S*I,Severity,Incidence,Province,Town,YearBuilt,Material,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,gas_natural,codine,Municipio,Superficie,Capitalidad,Año,CA,Autonomía,CP,Provincia,CPJ,Partido_Judicial
0,189311802,6,0,0.0,0.0,Friday,2010,2010-10-01,24,0.0,4,0,barcelona,sentmenat,2001,PE,160.0,117.831,4.000,0,0,False,1,8267.0,sentmenat,28.30,Sentmenat,NaN,9.0,Cataluña,8.0,Barcelona,813.0,Sabadell
1,188889914,6,0,0.0,0.0,Friday,2010,2010-10-01,22,0.0,4,0,barcelona,barcelona,1999,PE,160.0,1.292,0.025,0,0,False,1,8019.0,barcelona,98.21,Barcelona,NaN,9.0,Cataluña,8.0,Barcelona,811.0,Barcelona


In [46]:
#place Province and Town Columns at the end of the dataframe
cols = list(df_combined_mod.columns.values)
cols.pop(cols.index('Province'))
cols.pop(cols.index('Town'))
df_combined_mod = df_combined_mod[cols+['Province','Town']]
df_combined_mod.tail(2)

'Province'

'Town'

,PipeId,Inspections,No_Incidents,Risk_S*I/Inspections,leakage_estimate_factor,InspectionDay,InspectionYear,InspectionDate,MonthsLastRev,Risk_S*I,Severity,Incidence,YearBuilt,Material,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,gas_natural,codine,Municipio,Superficie,Capitalidad,Año,CA,Autonomía,CP,Provincia,CPJ,Partido_Judicial,Province,Town
1436390,56922456,1,0,0.0,0.0,Thursday,2020,2020-12-31,24,0.0,4,0,1993,PE,63.0,23.306,4.0,0,0,False,1,46070.0,betera,75.1,Bétera,NaN,10.0,Comunidad Valenciana,46.0,Valencia/València,4601.0,Llíria,valencia,betera
1436391,56922465,1,0,0.0,0.0,Thursday,2020,2020-12-31,24,0.0,4,0,1993,PE,63.0,1.778,4.0,0,0,False,1,46070.0,betera,75.1,Bétera,NaN,10.0,Comunidad Valenciana,46.0,Valencia/València,4601.0,Llíria,valencia,betera


In [47]:
df_mun.columns

Index(['codine', 'Municipio', 'Superficie', 'Capitalidad', 'Año', 'CA',
       'Autonomía', 'CP', 'Provincia', 'CPJ', 'Partido_Judicial'],
      dtype='object')

In [48]:
#deleting 'CP', 'Provincia', 'CPJ', 'Partido_Judicial', 'Capitalidad', 'Año', 'CA', 'codine', 'Municipio' from df_combined_mod dataframe
df_combined_mod = df_combined_mod.drop(['CP', 'Provincia', 'CPJ', 'Partido_Judicial', 'Capitalidad', 'Año', 'CA', 'codine', 'Municipio'], axis=1)
df_combined_mod.head(2)


,PipeId,Inspections,No_Incidents,Risk_S*I/Inspections,leakage_estimate_factor,InspectionDay,InspectionYear,InspectionDate,MonthsLastRev,Risk_S*I,Severity,Incidence,YearBuilt,Material,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,gas_natural,Superficie,Autonomía,Province,Town
0,189311802,6,0,0.0,0.0,Friday,2010,2010-10-01,24,0.0,4,0,2001,PE,160.0,117.831,4.000,0,0,False,1,28.30,Cataluña,barcelona,sentmenat
1,188889914,6,0,0.0,0.0,Friday,2010,2010-10-01,22,0.0,4,0,1999,PE,160.0,1.292,0.025,0,0,False,1,98.21,Cataluña,barcelona,barcelona


In [49]:
#count nulls in each column for df_combined_mod dataframe
df_combined_mod.isnull().sum()


PipeId                          0
Inspections                     0
No_Incidents                    0
Risk_S*I/Inspections            0
leakage_estimate_factor         0
InspectionDay                   0
InspectionYear                  0
InspectionDate                  0
MonthsLastRev                   0
Risk_S*I                        0
Severity                        0
Incidence                       0
YearBuilt                       0
Material                        0
Diameter                        0
Length                          0
Pressure                        0
NumConnections                  0
NumConnectionsUnder             0
BoolBridle                      0
gas_natural                     0
Superficie                 179644
Autonomía                  179644
Province                        0
Town                            0
dtype: int64

In [50]:
#delete column "Superficie" from df_combined_mod dataframe
df_combined_mod = df_combined_mod.drop(['Superficie'], axis=1)

In [51]:
#make a function for replacing the null values in df_combined_mod dataframe with a string of my choice
def replace_nulls(df, column, string):
    df[column] = df[column].fillna(string)
    return df

In [52]:
#for Province with value tarragona, replace null values in Autonomia with Cataluña

df_combined_mod.loc[df_combined_mod['Province'] == 'barcelona', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'barcelona', 'Autonomía'].fillna('Cataluña')
df_combined_mod.loc[df_combined_mod['Province'] == 'tarragona', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'tarragona', 'Autonomía'].fillna('Cataluña')
df_combined_mod.loc[df_combined_mod['Province'] == 'lleida', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'lleida', 'Autonomía'].fillna('Cataluña')
df_combined_mod.loc[df_combined_mod['Province'] == 'girona', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'girona', 'Autonomía'].fillna('Cataluña')



In [53]:
#count nulls in Autonomia column in df_combined_mod dataframe
df_combined_mod['Autonomía'].isnull().sum()

86507

In [54]:
#for Province with value "La Rioja", replace null values in Autonomía with "Rioja (La)"
df_combined_mod.loc[df_combined_mod['Province'] == 'la rioja', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'la rioja', 'Autonomía'].fillna('Rioja (La)')


In [55]:
#for Province with value Orense, replace null values in Autonomía with "Galicia"
df_combined_mod.loc[df_combined_mod['Province'] == 'orense', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'orense', 'Autonomía'].fillna('Galicia')
df_combined_mod.loc[df_combined_mod['Province'] == 'la coruna', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'la coruna', 'Autonomía'].fillna('Galicia')
df_combined_mod.loc[df_combined_mod['Province'] == 'pontevedra', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'pontevedra', 'Autonomía'].fillna('Galicia')
df_combined_mod.loc[df_combined_mod['Province'] == 'lugo', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'lugo', 'Autonomía'].fillna('Galicia')


In [56]:
#for Province with value Zaragoza, replace null values in Autonomía with "Aragón"
df_combined_mod.loc[df_combined_mod['Province'] == 'zaragoza', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'zaragoza', 'Autonomía'].fillna('Aragón')
df_combined_mod.loc[df_combined_mod['Province'] == 'huesca', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'huesca', 'Autonomía'].fillna('Aragón')

In [57]:
#for Province with value Toledo, replace null values in Autonomía with "Castilla-La Mancha"
df_combined_mod.loc[df_combined_mod['Province'] == 'toledo', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'toledo', 'Autonomía'].fillna('Castilla-La Mancha')
df_combined_mod.loc[df_combined_mod['Province'] == 'albacete', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'albacete', 'Autonomía'].fillna('Castilla-La Mancha')
df_combined_mod.loc[df_combined_mod['Province'] == 'ciudad real', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'ciudad real', 'Autonomía'].fillna('Castilla-La Mancha')
df_combined_mod.loc[df_combined_mod['Province'] == 'cuenca', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'cuenca', 'Autonomía'].fillna('Castilla-La Mancha')


In [58]:
#for Province with value Almeria, replace null values in Autonomía with "Andalucía"
df_combined_mod.loc[df_combined_mod['Province'] == 'almeria', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'almeria', 'Autonomía'].fillna('Andalucía')
df_combined_mod.loc[df_combined_mod['Province'] == 'sevilla', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'sevilla', 'Autonomía'].fillna('Andalucía')
df_combined_mod.loc[df_combined_mod['Province'] == 'jaen', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'jaen', 'Autonomía'].fillna('Andalucía')
df_combined_mod.loc[df_combined_mod['Province'] == 'cordoba', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'cordoba', 'Autonomía'].fillna('Andalucía')
df_combined_mod.loc[df_combined_mod['Province'] == 'cadiz', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'cadiz', 'Autonomía'].fillna('Andalucía')
df_combined_mod.loc[df_combined_mod['Province'] == 'granada', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'granada', 'Autonomía'].fillna('Andalucía')



In [59]:
#for Province with value Zamora, replace null values in Autonomía with "Castilla y León"
df_combined_mod.loc[df_combined_mod['Province'] == 'zamora', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'zamora', 'Autonomía'].fillna('Castilla y León')
df_combined_mod.loc[df_combined_mod['Province'] == 'segovia', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'segovia', 'Autonomía'].fillna('Castilla y León')
df_combined_mod.loc[df_combined_mod['Province'] == 'valladolid', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'valladolid', 'Autonomía'].fillna('Castilla y León')
df_combined_mod.loc[df_combined_mod['Province'] == 'leon', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'leon', 'Autonomía'].fillna('Castilla y León')
df_combined_mod.loc[df_combined_mod['Province'] == 'salamanca', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'salamanca', 'Autonomía'].fillna('Castilla y León')
df_combined_mod.loc[df_combined_mod['Province'] == 'burgos', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'burgos', 'Autonomía'].fillna('Castilla y León')
df_combined_mod.loc[df_combined_mod['Province'] == 'soria', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'soria', 'Autonomía'].fillna('Castilla y León')
df_combined_mod.loc[df_combined_mod['Province'] == 'palencia', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'palencia', 'Autonomía'].fillna('Castilla y León')


In [60]:
#for Province with value Madrid, replace null values in Autonomía with "Madrid (Comunidad de)"
df_combined_mod.loc[df_combined_mod['Province'] == 'madrid', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'madrid', 'Autonomía'].fillna('Madrid (Comunidad de)')


In [61]:
#for Province with value Alicante, replace null values in Autonomía with Comunidad Valenciana
df_combined_mod.loc[df_combined_mod['Province'] == 'alicante', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'alicante', 'Autonomía'].fillna('Comunidad Valenciana')
df_combined_mod.loc[df_combined_mod['Province'] == 'castellon', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'castellon', 'Autonomía'].fillna('Comunidad Valenciana')
df_combined_mod.loc[df_combined_mod['Province'] == 'valencia', 'Autonomía'] = df_combined_mod.loc[df_combined_mod['Province'] == 'valencia', 'Autonomía'].fillna('Comunidad Valenciana')


In [62]:
#count nulls in Autonomia column in df_combined_mod dataframe
df_combined_mod['Autonomía'].isnull().sum()

0

In [63]:
#droping df_mun from memory to free RAM
del df_mun
del df_inspection
del df_network

In [64]:
#change df_population POB32 to int32
df_population['POB20'] = df_population['POB20'].astype('int32')
df_population.dtypes

PROVINCIA    object
NOMBRE       object
POB20         int32
HOMBRES       int64
MUJERES       int64
dtype: object

In [71]:
df_combined_mod.dtypes

PipeId                              int32
Inspections                          int8
No_Incidents                        int32
Risk_S*I/Inspections              float32
leakage_estimate_factor           float32
InspectionDay                      object
InspectionYear                      int32
InspectionDate             datetime64[ns]
MonthsLastRev                       int32
Risk_S*I                          float32
Severity                             int8
Incidence                            int8
YearBuilt                            int8
Material                           object
Diameter                          float32
Length                            float32
Pressure                          float32
NumConnections                      int32
NumConnectionsUnder                 int32
BoolBridle                           bool
gas_natural                         uint8
Autonomía                          object
Province                           object
Town                              

In [66]:
#create function to change detypes in64 to int32 in a df
def change_dtypes(df):
    for col in df.columns:
        if df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
        elif df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
    return df

In [67]:
change_dtypes(df_combined_mod)

,PipeId,Inspections,No_Incidents,Risk_S*I/Inspections,leakage_estimate_factor,InspectionDay,InspectionYear,InspectionDate,MonthsLastRev,Risk_S*I,Severity,Incidence,YearBuilt,Material,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,gas_natural,Autonomía,Province,Town
0,189311802,6,0,0.0,0.0,Friday,2010,2010-10-01,24,0.0,4,0,2001,PE,160.0,117.831001,4.000,0,0,False,1,Cataluña,barcelona,sentmenat
1,188889914,6,0,0.0,0.0,Friday,2010,2010-10-01,22,0.0,4,0,1999,PE,160.0,1.292000,0.025,0,0,False,1,Cataluña,barcelona,barcelona
2,188889907,6,0,0.0,0.0,Friday,2010,2010-10-01,22,0.0,4,0,1992,PE,200.0,28.193001,0.025,1,0,False,1,Cataluña,barcelona,barcelona
3,188673355,6,0,0.0,0.0,Friday,2010,2010-10-01,22,0.0,4,0,1998,PE,200.0,66.259003,0.025,5,0,False,1,Cataluña,barcelona,barcelona
4,188558922,6,0,0.0,0.0,Friday,2010,2010-10-01,22,0.0,4,0,1999,PE,160.0,0.810000,0.025,0,0,False,1,Cataluña,barcelona,barcelona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1436387,56922448,1,0,0.0,0.0,Thursday,2020,2020-12-31,24,0.0,4,0,1993,PE,63.0,1.237000,4.000,0,0,False,1,Comunidad Valenciana,valencia,betera
1436388,56907425,1,0,0.0,0.0,Thursday,2020,2020-12-31,24,0.0,4,0,2000,PE,160.0,169.731995,4.000,1,0,False,1,Comunidad Valenciana,valencia,betera
1436389,7843615,1,0,0.0,0.0,Thursday,2021,2020-12-31,24,0.0,4,0,1995,FD,150.0,0.361000,0.025,0,0,True,1,Cataluña,barcelona,sabadell
1436390,56922456,1,0,0.0,0.0,Thursday,2020,2020-12-31,24,0.0,4,0,1993,PE,63.0,23.306000,4.000,0,0,False,1,Comunidad Valenciana,valencia,betera


In [68]:
#change df_combined_mod int64 columns to int32 to save memory
df_combined_mod['PipeId'] = df_combined_mod['PipeId'].astype('int32')
df_combined_mod['YearBuilt'] = df_combined_mod['YearBuilt'].astype('int8')
df_combined_mod['NumConnections'] = df_combined_mod['NumConnections'].astype('int32')
df_combined_mod['InspectionYear'] = df_combined_mod['InspectionYear'].astype('int32')
df_combined_mod['Severity'] = df_combined_mod['Severity'].astype('int8')
df_combined_mod['Incidence'] = df_combined_mod['Incidence'].astype('int8')
df_combined_mod['Inspections'] = df_combined_mod['Inspections'].astype('int8')



In [69]:
#delete columns from df_population dataframe
df_population.drop(['NOMBRE', 'HOMBRES', 'MUJERES'], axis=1, inplace=True)

In [70]:
#joining df_combined_mod and df_population on Province columns adding only POB20 column from df_population
df_combined_mod = pd.merge(df_combined_mod, df_population[['PROVINCIA','POB20']], how='left', left_on='Province', right_on='PROVINCIA')

MemoryError: Unable to allocate 2.04 GiB for an array with shape (274356552, 1) and data type float64

In [ ]:
df_combined_mod.columns

In [ ]:
#eda pandas profiling

# Further feature engineering

We want to also see materials available for the pipes as a hot encoded feature

In [ ]:
print (df_combined['Material'].unique())

In [ ]:
#Map df_combined Material column to the names of materials
df_combined_mod['Material'] = df_combined_mod['Material'].map({'PE': 'Polyethylene', 'AO': 'Acrylonitrile-Butadiene-Styrene', 'FD': 'Fiberglass-Reinforced Plastic', 
    'FG': 'Fiberglass', 'PN': 'Polypropylene', 'PA': 'Polyamide', 'FO': 'Flexible Polyolefin', 'FI': 'Flexible Polyvinyl Chloride', 'CU': 'Copper', 
    'PV': 'Polyvinylidene Fluoride', 'ZD': 'Zinc-Coated Steel', 'ZA': 'Zinc-Aluminum', 'CP': 'Cast Iron', 'CS': 'Cast Steel', 
    'ZC': 'Zinc-Coated Steel', 'ZM': 'Zinc-Magnesium','ZN': 'Zinc', 'AL': 'Aluminum', 'ZP': 'Zinc-Coated Steel', 'ZF': 'Zinc-Aluminum-Magnesium'})


In [ ]:
df_combined_mod.tail(1)

In [ ]:
#Hot enconde Material column in the df_combined_mod dataframe subset
df_combined_mod = pd.get_dummies(df_combined_mod, columns=['Material'], prefix = ['Material'])

In [ ]:
#allow to see all columns in pandas
pd.set_option('display.max_columns', None)
df_combined_mod.head(1)

In [ ]:
#deleting columns Material_Fiberglass, Material_Zinc_coated_steel, Material_Polyvinylidene_fluoride
df_combined_mod = df_combined_mod.drop(['Material_Fiberglass', 'Material_Zinc-Coated Steel', 'Material_Polyvinylidene Fluoride'], axis=1)

In [ ]:
#deleting columns Material_Flexible Polyolefin,Material_Flexible Polyvinyl Chloride, Material_Polyamide
df_combined_mod = df_combined_mod.drop(['Material_Flexible Polyolefin', 'Material_Flexible Polyvinyl Chloride', 'Material_Polyamide'], axis=1)
df_combined_mod.head(1)

In [ ]:
#delete outliers in year_built column to keep values only higher than 1960
df_combined_mod = df_combined_mod[df_combined_mod['YearBuilt'] > 1960]


In [ ]:
df_combined_mod.dtypes

In [ ]:
#eliminate values higher than 40 in MonthsLastRev column
df_combined_mod = df_combined_mod[df_combined_mod['MonthsLastRev'] < 40]

In [ ]:
#graph histogram of MonthsLastRev column
df_combined_mod['MonthsLastRev'].hist()

In [ ]:
#eliminate values higher than 400 in diameter column
df_combined_mod = df_combined_mod[df_combined_mod['Diameter'] < 400]

In [ ]:
#show histogram of diameter column
df_combined_mod['Diameter'].hist()

In [ ]:
#plot column Length
df_combined_mod['Length'].value_counts()

In [ ]:
#plot histogram of Length column
df_combined_mod['Length'].hist()

# Feature Engineering

In [ ]:
#create a new column that divides the diameter by the pressure and name it Relative Thickness
df_combined_mod['Relative_Thickness'] = df_combined_mod['Diameter'] / df_combined_mod['Pressure']

In [ ]:
#Create a new column that takes the inspecion year and substracts the YearBuilt column naming it Age of Pipe
df_combined_mod['Age_of_Pipe_upon_inspection'] = df_combined_mod['InspectionYear'] - df_combined_mod['YearBuilt']
df_combined_mod.head(1)

In [ ]:
#create a column that multiplies severity by incidence and name it severity_incidence
df_combined_mod['Severity_Incidence'] = df_combined_mod['Severity'] * df_combined_mod['Incidence']

In [ ]:
#delete column severity
df_combined_mod = df_combined_mod.drop(['Severity'], axis=1)
df_combined_mod.head(1)


In [ ]:
df_combined_mod.dtypes

In [ ]:
#hot encode severity_incidence column in the df_combined_mod dataframe
df_combined_mod = pd.get_dummies(df_combined_mod, columns=['Severity_Incidence'], prefix = ['Severity_Incidence'])
df_combined_mod.head(1)

In [ ]:
#change name of Severity_Incidence_0 column to Severity_0
df_combined_mod = df_combined_mod.rename(columns={'Severity_Incidence_0': 'Severity_0'})
#change name of Severity_Incidence_1 column to Severity_low
df_combined_mod = df_combined_mod.rename(columns={'Severity_Incidence_1': 'Severity_low'})
#change name of Severity_Incidence_2 column to Severity_medium
df_combined_mod = df_combined_mod.rename(columns={'Severity_Incidence_2': 'Severity_medium'})
#change name of Severity_Incidence_3 column to Severity_high
df_combined_mod = df_combined_mod.rename(columns={'Severity_Incidence_3': 'Severity_high'})
df_combined_mod.head(1)


In [ ]:
df_combined_mod.shape

In [ ]:
#General number of high severity per year
df_combined_mod.groupby('InspectionYear')['Severity_high'].value_counts()

In [ ]:
df_combined_mod.dtypes

# Exporting the new dataset for a backup

In [ ]:
#point to the folder where the data is stored
os.chdir(r"C:\Users\pedro\datathon")
#export the dataframe to a csv file
df_combined_mod.to_csv('df_combined_mod.csv', index=False)

# Plotting EDA for new dataset

In [ ]:
#pandas profiling on the df_combined_mod dataframe
profile = ProfileReport(df_combined_mod, title='Pandas Profiling Report', html={'style':{'full_width':True}})
profile

# WOULD BE NICE TO KNOW

- what do we do with categoricals
    province and town, are they relevant?
- how toi interpret the length?
- an extra dataset would be great to create more features 
- should we create polinomial features

In [ ]:
#create polinomial features 
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)
poly_data = poly.fit_transform(df_combined_mod[['Relative_Thickness', 'Age_of_Pipe_upon_inspection']])
poly_columns = poly.get_feature_names(['Relative_Thickness', 'Age_of_Pipe_upon_inspection'])
df_poly = pd.DataFrame(poly_data, columns=poly_columns)
df_poly = df_poly.drop(['1'], axis=1)
df_combined_mod = pd.concat([df_combined_mod, df_poly], axis=1)
df_combined_mod.head(1)